# 🚀 Deploy Your Fine-tuned RAFT Model with Amazon SageMaker

## Install Dependencies

In [ ]:
%pip install -Uq sagemaker

In [ ]:
import json
import sagemaker
import boto3
from typing import List, Dict
from datetime import datetime
from sagemaker.huggingface import (
    HuggingFaceModel, 
    get_huggingface_llm_image_uri
)
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

In [ ]:
boto_region = boto3.Session().region_name
session = sagemaker.session.Session(boto_session=boto3.Session(region_name=boto_region))
role = sagemaker.get_execution_role()

## Deploy using DJL-Inference Container

The [Deep Java Library (DJL) Large Model Inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-container-docs.html) containers are specialized Docker containers designed to facilitate the deployment of large language models (LLMs) on Amazon SageMaker. These containers integrate a model server with optimized inference libraries, providing a comprehensive solution for serving LLMs. 

**Key Features of DJL LMI Containers:**

* __Optimized Inference Performance__: Support for popular model architectures like DeepSeek, Mistral, Llama, Falcon and many more..
* __Integration with Inference Libraries__: Seamless integration with libraries such as vLLM, TensorRT-LLM, and Transformers NeuronX.
* __Advanced Capabilities__: Features like continuous batching, token streaming, quantization (e.g., AWQ, GPTQ, FP8), multi-GPU inference using tensor parallelism, and support for LoRA fine-tuned models.

**Benefits for Deploying LLMs with DJL-LMI on Amazon SageMaker:**

* __Simplified Deployment__: DJL LMI containers offer a low-code interface, allowing users to specify configurations like model parallelization and optimization settings through a configuration file. 
* __Performance Optimization__: By leveraging optimized inference libraries and techniques, these containers enhance inference performance, reducing latency and improving throughput.
* __Scalability__: Designed to handle large models that may not fit on a single accelerator, enabling efficient scaling across multiple GPUs or specialized hardware like AWS Inferentia.

In [ ]:
lmi_container_uri = f"763104351884.dkr.ecr.{boto_region}.amazonaws.com/djl-inference:0.31.0-lmi13.0.0-cu124"

Choose an appropriate model name and endpoint name when hosting your model.

In [ ]:
model_timestamp = datetime.now().strftime('%y%m%d-%H%M%S')

base_model_name = f"llama-3-1-8B-base-lmi-{model_timestamp}"
tuned_model_name = f"llama-3-1-8B-tuned-lmi-{model_timestamp}"

base_endpoint_name = f"{base_model_name}-ep"
tuned_endpoint_name = f"{tuned_model_name}-ep"

print(f"base: \n{base_endpoint_name}")
print(f"tuned: \n{tuned_endpoint_name}")

Create a new [SageMaker Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)

> ⚠ Swap `HF_MODEL_ID` with another model tag if you want to compare against a different base model.
>
> Gated models will require you to supply a HuggingFace API Token via the `HF_TOKEN: "hf_..."` parameter.

In [ ]:
#set these to either S3 paths or HuggingFace model tags

BASE_MODEL_ARTIFACTS =  "<<PATH_TO_YOUR_BASE_MODEL>>"
TUNED_MODEL_ARTIFACTS = "<<PATH_TO_YOUR_TUNED_MERGED_MODEL>>"

In [ ]:
base_model = sagemaker.Model(
    image_uri=lmi_container_uri,
    env={
        "HF_MODEL_ID": BASE_MODEL_ARTIFACTS,
        "OPTION_MAX_MODEL_LEN": "5000",
        "OPTION_GPU_MEMORY_UTILIZATION": "0.95",
        "OPTION_ENABLE_STREAMING": "false",
        "OPTION_ROLLING_BATCH": "auto",
        "OPTION_MODEL_LOADING_TIMEOUT": "3600",
        "OPTION_PAGED_ATTENTION": "false",
        "OPTION_DTYPE": "fp16",
    },
    role=role,
    name=base_model_name,
    sagemaker_session=sagemaker.Session()
)

In [ ]:
tuned_model = sagemaker.Model(
    image_uri=lmi_container_uri,
    env={
        "HF_MODEL_ID": TUNED_MODEL_ARTIFACTS,
        "OPTION_MAX_MODEL_LEN": "5000",
        "OPTION_GPU_MEMORY_UTILIZATION": "0.95",
        "OPTION_ENABLE_STREAMING": "false",
        "OPTION_ROLLING_BATCH": "auto",
        "OPTION_MODEL_LOADING_TIMEOUT": "3600",
        "OPTION_PAGED_ATTENTION": "false",
        "OPTION_DTYPE": "fp16",
    },
    role=role,
    name=tuned_model_name,
    sagemaker_session=sagemaker.Session()
)

🚀 Deploy. Please wait for the endpoint to be `InService` before running inference against it!

In [ ]:
base_model_predictor = base_model.deploy(
    endpoint_name=base_endpoint_name,
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    container_startup_health_check_timeout=600,
    wait=True
)
print(f"\nYour BASE Endpoint: {base_endpoint_name} is now deployed! 🚀")

In [ ]:
tuned_model_predictor = tuned_model.deploy(
    endpoint_name=tuned_endpoint_name,
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    container_startup_health_check_timeout=600,
    wait=True
)
print(f"\nYour TUNED Endpoint: {tuned_endpoint_name} is now deployed! 🚀")

### Inference with SageMaker SDK

SageMaker python sdk simplifies the inference construct using `sagemaker.Predictor` method.

Llama 3 utilizes the following prompt format:


```json
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an assistant for question-answering tasks. Answer the following question in 5 sentences using the provided context. If you don't know the answer, just say "I don't know.".

<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Context: {CONTEXT}

Question: {QUESTION} 
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
```

In [ ]:
def format_messages(messages: list[dict[str, str]]) -> list[str]:
    """
    Format messages for Llama 3+ chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    # auto assistant suffix
    # messages.append({"role": "assistant"})
    
    output = "<|begin_of_text|>"
    # Adding an inferred prefix
    system_prefix = f"\n\nCutting Knowledge Date: December 2024\nToday Date: {datetime.now().strftime('%d %b %Y')}\n\n"
    for i, entry in enumerate(messages):
        output += f"<|start_header_id|>{entry['role']}<|end_header_id|>"
        if entry['role'] == 'system':
            output += f"{system_prefix}{entry['content']}<|eot_id|>"
        elif entry['role'] != 'system' and 'content' in entry:
            output += f"\n\n{entry['content']}<|eot_id|>"
    output += "<|start_header_id|>assistant<|end_header_id|>\n"
    return output


def send_prompt(predictor, messages, parameters):
    # convert u/a format 
    frmt_input = format_messages(messages)
    payload = {
        "inputs": frmt_input,
        "parameters": parameters
    }
    response = predictor.predict(payload)
    return response

In [ ]:
def build_messages(data):
    system_content = f"""You are an assistant for question-answering tasks. Answer the following question in 5 sentences using the provided context. If you don't know the answer, just say "I don't know."."""
    user_content = f"""
        Context: {data["context"]} 
        
        Question: {data["question"]}
        """

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
    
    return messages

We can continue to use a simple `List[Dict[str, str]]` format to chat and simplify `system`, `user` and `assistant` chat transcripts.

In [ ]:
from datasets import load_dataset, concatenate_datasets

test_dataset = load_dataset("json", data_files="datasets/raft/test/test.json", split="train")

In [ ]:
test_item = test_dataset[0]
test_item

reloading the predictors from endpoint names in case we are working with existing endpoints

In [ ]:
base_predictor = sagemaker.Predictor(
    endpoint_name="<<YOUR_BASE_ENDPOINT_NAME>", #base_endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

tuned_predictor = sagemaker.Predictor(
    endpoint_name="<<YOUR_TUNED_ENDPOINT_NAME>>", #tuned_endpoint_name,
    sagemaker_session=session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [ ]:
%%time

messages = build_messages(test_item)

base_response = send_prompt(
    base_predictor,
    messages,
    parameters={
        "temperature": 0.9, 
        "max_new_tokens": 512,
        "top_p": 0.9
    }
)

tuned_response = send_prompt(
    tuned_predictor,
    messages,
    parameters={
        "temperature": 0.9, 
        "max_new_tokens": 512,
        "top_p": 0.9
    }
)

print(f"""
    ============== Question ============
    {test_item["question"]}

    ========= Baseline Answer ==========
    {base_response['generated_text']}
    
    ========= Generated Answer =========
    {tuned_response['generated_text']}

    ======== Ground Truth Answer =======
    {test_item["synthetic_answer"]}
    """
)

## Evaluate your results

In this section, you will build a dataset of evaluation data. The `MAX_EVALUATIONS` value will limit the scope of the evaluation and the time it takes to complete.

Since there are pure distractor documents from splitting our training dataset, we will remove them during the evaluation.

In [ ]:
questions = []
oracle_context = []
test_context = []
ground_truth = []
base_predictions = []
sft_predictions = []

#evaluation_data = {}

MAX_EVALUATIONS = -1 #set to -1 to run the entire dataset. WARNING: THIS WILL TAKE A LONG TIME

if MAX_EVALUATIONS > -1:
    print(f"MAX_EVALUATIONS set, reducing input to {MAX_EVALUATIONS} items.")
else:
    MAX_EVALUATIONS = len(test_dataset)

for idx, test_item in enumerate(test_dataset.select(range(MAX_EVALUATIONS))):
    
    if test_item["distracted"] == True:
        continue #skip distractor docs

    
    messages = build_messages(test_item)
    
    base_response = send_prompt(
        base_predictor,
        messages,
        parameters={
            "temperature": 0.9, 
            "max_new_tokens": 512,
            "top_p": 0.9
        }
    )
    
    sft_response = send_prompt(
        tuned_predictor,
        messages,
        parameters={
            "temperature": 0.9, 
            "max_new_tokens": 512,
            "top_p": 0.9
        }
    )

    # Define the candidate predictions and reference sentences

    # evaluation_data.append({
    #     "ground_truth": test_item["ANSWER"],
    #     "base": base_response['generated_text'],
    #     "tuned": sft_response['generated_text'],
    #     "test_context": test_item["CONTEXT"],
    #     "oracle_context": test_item["ORACLE"]
    # })
    
    ground_truth.append(test_item["synthetic_answer"])
    base_predictions.append(base_response['generated_text'])
    sft_predictions.append(sft_response['generated_text'])

    print(f"{idx} of {MAX_EVALUATIONS}", end="\r")
    

evaluation_dataset = Dataset.from_dict(
    {
        "ground_truth": ground_truth, 
        "base": base_predictions, 
        "tuned": sft_predictions}
)
evaluation_dataset.to_json(f"./eval.json", orient="records")

In [ ]:
ground_truth = []
base_predictions = []
sft_predictions = []

for eval_item in evaluation_dataset:

    ground_truth.append(eval_item["ground_truth"])
    base_predictions.append(eval_item['base'])
    sft_predictions.append(eval_item['tuned'])


base_bleu_results = bleu.compute(predictions=base_predictions, references=ground_truth)
base_rouge_results = rouge.compute(predictions=base_predictions, references=ground_truth)

# Compute the BLEU score
sft_bleu_results = bleu.compute(predictions=sft_predictions, references=ground_truth)
sft_rouge_results = rouge.compute(predictions=sft_predictions, references=ground_truth)

base_scores = (base_bleu_results | base_rouge_results)
sft_scores = (sft_bleu_results | sft_rouge_results)


# base_scores.append(base_bleu_results | base_rouge_results)
# sft_scores.append(sft_bleu_results | sft_rouge_results)
print("=======BASE MODEL=======")
print(base_scores)
print("=======TUNED MODEL=======")
print(sft_scores)

In [ ]:
import pandas as pd
data = {'dimension':[], 'base': [], 'tuned': [], 'delta': [], 'delta_percent': []}

for key in base_scores.keys():
    if key in ["length_ratio","precisions","brevity_penalty","translation_length","reference_length"]:
        continue
        
    delta = sft_scores[key]-base_scores[key]
    delta_percent = (delta/base_scores[key])*100
    
    data['dimension'].append(key)
    data['base'].append(base_scores[key])
    data['tuned'].append(sft_scores[key])
    data['delta'].append(delta)
    data['delta_percent'].append(delta_percent)
    
df = pd.DataFrame(data)

df